# Search for accusing tweets

In this notebook we will search for accusing tweets. We mean by this replies of this kind:

- "you are a racist"
- "You are a misoginist"
- "you hate black people" 

and so on

In [1]:
%load_ext autoreload
%autoreload 2
from mongoengine import connect
from tweepyrate import create_apps
from hate_collector.models import Tweet

client = connect("hatespeech")
db = client["hatespeech"]

Let's look for tweets containing "sos machista"

In [26]:
accusing_tweets = Tweet.objects(__raw__={
    "$text": {"$search": "machista"},
    "in_reply_to_status_id": {"$ne": None},
    "$expr": {"$lt": [{"$strLenCP": "$text"}, 80]},
})

print(len(accusing_tweets))
for tweet in accusing_tweets[:20]:
    print(f"-- {tweet.text}")

2261
-- @jmarialg más machistas que nunca, es la contrarreacción machista
-- @ExpositoCOPE Machista, a eso mi madre lo llama machista.
-- @FemiCadiz Machista coño,VIOLENCIA MACHISTA
-- @JonHeguier Y más machista!!!!! Te faltó decir que es machista en extremo.
-- ¡MACHISTA!
-- machista*
-- @Dayana21586226 es machista, o acaso los machistas son buenas personas?
-- Y sobre todo es machista.
-- @j7dios2018 Maldito machista, ojala se mueran todos los machistas
-- @gerardobellom Michista! Machista! Machista! https://t.co/kOIEmfhvi7
-- @Canarixn_ Se puede ser más machista,vaya comentario mas machista y bajuno
-- @anarcogatinha Machista
-- @milfodidas -machista
-- @eduardofusion machista
-- @o_antagonista MACHISTA!
-- @doriangraypic machistA
-- @karlaraeguia Machista
-- @polrdgrimes Qué machista eres
-- @BluRadioCo qué machista
-- @jeancabrero Machista


How many of these replies have their original tweet in db?


In [48]:
upstream_ids = set(tweet.in_reply_to_status_id for tweet in accusing_tweets)

print(f"There are {len(upstream_ids)} tweets to be retrieved")

upstream_tweets = Tweet.objects(id__in=upstream_ids)
print(f"There are {upstream_tweets.count()} in DB")

There are 2025 tweets to be retrieved
There are 1953 in DB


I will look only for those which don't have any urls inside


In [51]:

upstream_tweets = Tweet.objects(__raw__={
    "_id": {"$in": list(upstream_ids)},
    "entities.urls": {"$size": 0},
})

print(f"There are {upstream_tweets.count()} without urls")

for tweet in upstream_tweets:
    print(f"\n-- {tweet.text}")

There are 1509 without urls

-- FELIZ ANO NOVO POVO

-- @Pedro374181781 #RAEconsultas Si se trata de ungrupo mixto, solo puede hacer la referencia conjunta el masculino: «los feministas».

-- "Sex education" puede ser mejor o peor que otras series, peeo por lo menos tiene un personaje gay NO HETERONORMATIVO, que eso es algo que nunca pasa...

-- Cuando a algún hombre que conozcáis, o a vosotros mismos os caiga una denuncia falsa muy rica por parte de una mujer sin escrúpulos, le dais las gracias a los millones de votantes del PSOE. Yo he intentado cambiar eso con mi voto. Quería igualdad para todas las personas. Y quiero

-- La prostitución lleva "debatiéndose" desde los 70. Ahora ya es hora de debatir cómo abolirla y dejar de defender posturas que favorecen a los hombres y al sistema capitalista neoliberal.

-- @Jurengue Comia fodase

-- @leoneltudo se hace la asqueada y se garcho a un pibe estando casado, flaca no da.

-- Y avisenle a wendi que pague sus deudas , xq es fácil cagar al

In [44]:
tweet = Tweet.objects.search_text('"Quien" fuera "Presidenta de Madrid"')[0]

In [47]:
Tweet.objects()

{'_cls': 'Tweet',
 '_dynamic_lock': False,
 '_fields_ordered': ('id',
  'text',
  'created_at',
  'id_str',
  'full_text',
  'truncated',
  'display_text_range',
  'entities',
  'source',
  'in_reply_to_status_id',
  'in_reply_to_status_id_str',
  'in_reply_to_user_id',
  'in_reply_to_user_id_str',
  'in_reply_to_screen_name',
  'user',
  'geo',
  'coordinates',
  'place',
  'contributors',
  'is_quote_status',
  'retweet_count',
  'favorite_count',
  'favorited',
  'retweeted',
  'possibly_sensitive',
  'possibly_sensitive_appealable',
  'lang'),
 'id_str': '1214596613152350208',
 'full_text': 'Quien fuera Presidenta de Madrid. La mujer que pudo suceder a Rajoy. Al menos esto no se puede considerar puerta giratoria...  https://t.co/Jbs7cXhluH',
 'truncated': False,
 'display_text_range': [0, 150],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/Jbs7cXhluH',
    'expanded_url': 'https://look.okdiario.com/famosos/cristina-cifuentes-